# Camada Gold - BI Executivo de E-commerce

> **POC:** E-commerce Executive BI  
> **Objetivo:** Construção da camada Gold para consumo em Power BI

Este notebook documenta a construção da **camada Gold** utilizada na POC de **Business Intelligence Executivo**, a partir da base curada previamente.

O foco desta etapa é consolidar informações operacionais e financeiras em um dataset único, com granularidade adequada para consumo em ferramentas de BI (e.g. Power BI), permitindo análises rápidas e consistentes sobre o desempenho do e-commerce.

---

## Contexto

Os dados utilizados neste notebook são derivados diretamente da camada de
curadoria (Silver), construída no notebook [`01_curadoria_sql.ipynb`](notebooks/01_curadoria_sql.ipymb).

Toda a lógica de padronização, validação e agregação intermediária já foi realizada anteriormente. Neste notebook, **não há tratamento de dados brutos**, apenas consolidação analítica orientada a consumo.

---

## Objetivo da Camada Gold

Criar um dataset com:
- Granularidade **1 linha por pedido (`order_id`)**
- Métricas financeiras e operacionais consolidadas
- Informações de cliente, localização e categoria
- Flags analíticas que simplificam filtros e KPIs no BI

O resultado final é a view `gold_orders_enriched`, que serve como fonte direta para o dashboard executivo no Power BI.

## Conexão e Verificação Inicial

Nesta seção são realizadas a importação das bibliotecas necessárias, a conexão com o banco de dados DuckDB e uma verificação inicial das tabelas e views disponíveis no ambiente analítico.

In [1]:
import sys; sys.path.insert(0, "..")
from src.paths import PROCESSED_DATA
import duckdb

In [2]:
con = duckdb.connect(database=str(PROCESSED_DATA), read_only=False)
con.execute("SELECT 1").fetchone()

(1,)

In [3]:
# listar tabelas e views
con.execute("""
SELECT table_schema, table_name, table_type
FROM information_schema.tables
WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
ORDER BY table_type, table_schema, table_name;
""").df()

,table_schema,table_name,table_type
0,main,category_translation,BASE TABLE
1,main,customers,BASE TABLE
2,main,geolocation,BASE TABLE
3,main,order_items,BASE TABLE
4,main,order_payments,BASE TABLE
5,main,order_reviews,BASE TABLE
6,main,orders,BASE TABLE
7,main,products,BASE TABLE
8,main,sellers,BASE TABLE
9,main,curated_category_translation,VIEW


## Construção da Camada Gold — `gold_orders_enriched`

Nesta etapa é criada a view `gold_orders_enriched`, responsável por consolidar informações de pedidos, itens agregados, pagamentos e atributos de cliente em um único dataset analítico.

A granularidade adotada é **1 linha por pedido**, o que facilita:
- Modelagem no Power BI
- Criação de KPIs executivos
- Análises temporais e geográficas


In [4]:
con.execute("""
CREATE OR REPLACE VIEW gold_orders_enriched AS
WITH order_category AS (
  SELECT
    x.order_id,
    arg_max(pr.product_category_en, x.cnt) AS main_category_en
  FROM (
    SELECT
      order_id,
      product_id,
      COUNT(*) AS cnt
    FROM curated_order_items
    GROUP BY order_id, product_id
  ) x
  LEFT JOIN curated_products pr
    ON x.product_id = pr.product_id
  GROUP BY x.order_id
)
SELECT
  -- chaves
  o.order_id,
  o.customer_id,

  -- status
  o.order_status,

  -- flags BI (para filtro e KPI limpo)
  CASE WHEN o.order_status = 'delivered' THEN 1 ELSE 0 END AS is_completed_order,
  CASE
    WHEN COALESCE(i.total_gmv, 0) > 0 AND COALESCE(p.total_payment_value, 0) > 0 THEN 1
    ELSE 0
  END AS has_financials,

  -- timestamps
  o.order_purchase_ts,
  o.order_approved_ts,
  o.order_delivered_carrier_ts,
  o.order_delivered_customer_ts,
  o.order_estimated_delivery_ts,

  -- datas
  o.purchase_date,
  o.delivered_date,
  o.estimated_delivery_date,

  -- calendário
  EXTRACT(year  FROM o.purchase_date) AS purchase_year,
  EXTRACT(month FROM o.purchase_date) AS purchase_month,

  -- logística
  o.days_to_deliver,
  o.delay_days,
  o.delay_days_pos,
  o.early_days_pos,
  o.is_delivered,
  o.is_delayed,

  -- itens
  COALESCE(i.items_count, 0) AS items_count,
  COALESCE(i.unique_products, 0) AS unique_products,
  COALESCE(i.unique_sellers, 0) AS unique_sellers,
  COALESCE(i.total_price, 0) AS total_price,
  COALESCE(i.total_freight, 0) AS total_freight,
  COALESCE(i.total_gmv, 0) AS total_gmv,

  -- pagamentos
  COALESCE(p.total_payment_value, 0) AS total_payment_value,
  p.max_installments,
  COALESCE(p.num_payments, 0) AS num_payments,
  p.payment_type_main,

  -- cliente
  c.customer_state,
  c.customer_city,
  c.customer_zip_code_prefix,

  -- geo
  g.latitude  AS customer_latitude,
  g.longitude AS customer_longitude,

  -- categoria
  COALESCE(oc.main_category_en, 'unknown') AS main_category_en

FROM curated_orders o
LEFT JOIN curated_items_by_order i
  ON o.order_id = i.order_id
LEFT JOIN curated_payments_by_order p
  ON o.order_id = p.order_id
LEFT JOIN curated_customers c
  ON o.customer_id = c.customer_id
LEFT JOIN curated_geolocation_by_zip g
  ON c.customer_zip_code_prefix = g.zip_code_prefix
LEFT JOIN order_category oc
  ON o.order_id = oc.order_id

-- Contenção: remove registros inconsistentes para consumo executivo no BI
WHERE
  o.order_status = 'delivered'
  AND COALESCE(i.total_gmv, 0) > 0
  AND COALESCE(p.total_payment_value, 0) > 0
  AND p.payment_type_main IS NOT NULL
  AND p.payment_type_main <> 'not_defined';
""")

### Flags Analíticas

Foram adicionadas duas flags à camada Gold com o objetivo de simplificar o
consumo no BI e reduzir ruído em análises executivas:

- **`is_completed_order`**  
  Indica pedidos efetivamente entregues (`order_status = 'delivered'`), permitindo
  a construção de KPIs focados apenas em operações concluídas.

- **`has_financials`**  
  Indica pedidos com valores financeiros consistentes (GMV e pagamento maiores
  que zero), evitando a inclusão de pedidos cancelados ou incompletos em análises
  de receita.


## Validações da Camada Gold

Após a criação da view, são realizadas validações básicas para garantir:

- Integridade da granularidade (1 linha por pedido)
- Coerência das flags analíticas
- Sanidade das métricas financeiras e temporais

In [5]:
# Integridade da granularidade
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT order_id) AS distinct_orders,
  COUNT(*) - COUNT(DISTINCT order_id) AS dup_orders
FROM gold_orders_enriched;
""").df()

,rows,distinct_orders,dup_orders
0,96477,96477,0


In [6]:
# Coerência das flags analíticas
con.execute("""
SELECT
  SUM(is_completed_order) AS completed_orders,
  SUM(has_financials) AS orders_with_financials,
  COUNT(*) AS total_orders
FROM gold_orders_enriched;
""").df()

,completed_orders,orders_with_financials,total_orders
0,96477.0,96477.0,96477


In [7]:
# zeros vs flags
con.execute("""
SELECT
  SUM(CASE WHEN total_gmv = 0 THEN 1 ELSE 0 END) AS zero_gmv_orders,
  SUM(CASE WHEN total_payment_value = 0 THEN 1 ELSE 0 END) AS zero_payment_orders,
  SUM(CASE WHEN has_financials = 0 THEN 1 ELSE 0 END) AS no_financials_flagged
FROM gold_orders_enriched;
""").df()

,zero_gmv_orders,zero_payment_orders,no_financials_flagged
0,0.0,0.0,0.0


## Estado Final do Ambiente Analítico

A seguir é apresentada a listagem final de tabelas e views disponíveis após a criação da camada Gold.

In [8]:
con.execute("""
SELECT table_schema, table_name, table_type
FROM information_schema.tables
WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
ORDER BY table_type, table_schema, table_name;
""").df()

,table_schema,table_name,table_type
0,main,category_translation,BASE TABLE
1,main,customers,BASE TABLE
2,main,geolocation,BASE TABLE
3,main,order_items,BASE TABLE
4,main,order_payments,BASE TABLE
5,main,order_reviews,BASE TABLE
6,main,orders,BASE TABLE
7,main,products,BASE TABLE
8,main,sellers,BASE TABLE
9,main,curated_category_translation,VIEW


In [9]:
con.close()

## Conclusão

A camada Gold criada e validada com sucesso, estando pronta para consumo em ferramentas de Business Intelligence.

O próximo passo consiste na modelagem e visualização dos dados no Power BI, utilizando a view `gold_orders_enriched` como fonte principal.